In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import torch, os
import torch.nn as nn

In [2]:
print(torch.__version__)
print(tf.__version__)
tf.compat.v1.disable_eager_execution()
tf.disable_v2_behavior()  # change tf version 2 -> version1 code.

1.6.0
2.3.0
Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
def samplePoint(n):
    """
    :param n: Total number of data-points
    :return: A tuple(X,y) where X-> [n,2] and y -> [n]
    """
    radius = np.random.uniform(low=0, high=2, size=n).reshape(-1,1) # uniform radius between 0 and 2
    angle = np.random.uniform(low=0, high=2*np.pi, size=n).reshape(-1,1) #uniform angle
    x1 = radius*np.cos(angle)
    x2 = radius*np.sin(angle)
    y = (radius<1).astype(int).reshape(-1)
    x = np.concatenate([x1, x2], axis=1)
    return x,y

In [4]:
# Generating the data
X_tr, Y_tr = samplePoint(10000)
X_val, Y_val = samplePoint(500)

print(X_tr.shape, Y_tr.shape)

(10000, 2) (10000,)


In [5]:
# Initialize the FlieWriters for "training" and "validation" routines
#trainWriterTf = tf.summary.create_file_writer("../log/train") 
trainWriterTf = tf.summary.FileWriter("../log/train")
# weightInit = tf.glorot_uniform_initializer() -> tf.compat.v1.keras.initializers.glorot_uniform()
# lastest using tf.keras.initializers.GlorotUniform() method 
def buildGraph(nUnits=12, nLayers=2, weightInit=tf.glorot_uniform_initializer(), biasInit=None, activation=tf.nn.relu, learningRate=1e-3
              ):
    X = tf.compat.v1.placeholder(tf.float32, shape=(None,2), name='X')
    Y = tf.compat.v1.placeholder(dtype=tf.int64, shape=[None], name='Y')
    gs = tf.compat.v1.train.get_or_create_global_step()
    
    with tf.compat.v1.variable_scope("network", reuse=tf.compat.v1.AUTO_REUSE):
        net = X
        for layer in range(nLayers):
            net = tf.compat.v1.layers.dense(net, units=nUnits, name="LAYER-{}".format(layer+1),
                                 activation=activation,
                                 kernel_initializer=weightInit, bias_initializer=biasInit)
            logits = tf.compat.v1.layers.dense(net, units=nUnits, name="LAYER-Last", activation=None,
                                 kernel_initializer=weightInit, bias_initializer=biasInit)
            loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=Y)
            acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits,1), Y),tf.float32))
            
            opt = tf.compat.v1.train.AdamOptimizer(learning_rate=learningRate)
            train = opt.minimize(loss, global_step=gs)
            
            # Evaluating the gradients to log in TensorBoard
            grads = opt.compute_gradients(loss)
            for grad in grads: tf.summary.histogram("{}-grad".format(grad[1].name), grad[0])
            
        # Add "loss" and "acc" as scalar summaries
        tf.summary.scalar("loss", tf.reduce_mean(loss))
        tf.summary.scalar("accuracy", acc)
        
        # Collect all trainable variables
        allWeights = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES)
        for weight in allWeights: tf.summary.histogram(weight.name, weight)
        
        # Merge all summaries into a single op
        summary = tf.compat.v1.summary.merge_all()
        
        return {
            "X": X, "Y": Y, "train": train, "loss": tf.reduce_mean(loss), "acc": acc,
            "gs": gs, "summ": summary
        }

In [6]:
def startScalarTraining(epochs=100):
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        for epoch in range(epochs):
            # *train* operands // ops is operations...
            gs, _, summary = sess.run([ops["gs"], ops["train"], ops["summ"]], feed_dict={ops["X"]: X_tr, ops["Y"]: Y_tr})
            #gs, _, summary = buildGraph(X_tr, Y_tr)
            trainWriterTf.add_summary(summary, epoch) #Logging the summary in the event file
            trainWriterTf.flush() # Write to disk

# Reset graph
tf.compat.v1.reset_default_graph()

# Build the graph and start training
ops = buildGraph(weightInit=tf.random_uniform_initializer(minval=-0.01, maxval=0.01))
startScalarTraining(epochs=200)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Summary name network/LAYER-1/kernel:0-grad is illegal; using network/LAYER-1/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/bias:0-grad is illegal; using network/LAYER-1/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/kernel:0-grad is illegal; using network/LAYER-Last/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/bias:0-grad is illegal; using network/LAYER-Last/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/kernel:0-grad is illegal; using network/LAYER-1/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/bias:0-grad is illegal; using network/LAYER-1/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/kernel:0-grad is illegal; using network/LAYER-Last/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/bias

In [99]:
# Random uniform initialization of biases and using Glorot initialization for Weight matrices.
trainWriterTf = tf.summary.FileWriter("../log/train_5_layers")

# Reset graph
tf.reset_default_graph()

# Build th egraph and start training
ops = buildGraph(nUnits=12, nLayers=5, biasInit=tf.random_uniform_initializer(minval=-0.2, maxval=0.2))
startScalarTraining(epochs=200)

INFO:tensorflow:Summary name network/LAYER-1/kernel:0-grad is illegal; using network/LAYER-1/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/bias:0-grad is illegal; using network/LAYER-1/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/kernel:0-grad is illegal; using network/LAYER-Last/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/bias:0-grad is illegal; using network/LAYER-Last/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/kernel:0-grad is illegal; using network/LAYER-1/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/bias:0-grad is illegal; using network/LAYER-1/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/kernel:0-grad is illegal; using network/LAYER-Last/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/bias:0-grad is illegal; using network/LAYER-Last/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-2/kernel:0-grad is il

In [7]:
# Using Different Activation function

#Logging for all Activations
#0.1
trainWriterTf = tf.summary.FileWriter("../log/trainSigmoid")

# Reset graph
tf.reset_default_graph()

# Build the graph and start training
ops = buildGraph(biasInit=tf.random_uniform_initializer(minval=-0.2, maxval=0.2), activation=tf.nn.sigmoid)
startScalarTraining(epochs=200)

#0.2
trainWriterTf = tf.summary.FileWriter("../log/trainTanh")

# Reset graph
tf.reset_default_graph()

# Build the graph and start training
ops = buildGraph(biasInit=tf.random_uniform_initializer(minval=-0.2, maxval=0.2), activation=tf.nn.tanh)
startScalarTraining(epochs=200)

#0.3
trainWriterTf = tf.summary.FileWriter("../log/trainLrelu")

# Reset graph
tf.reset_default_graph()

# Build the graph and start training
ops = buildGraph(biasInit=tf.random_uniform_initializer(minval=-0.2, maxval=0.2), activation=tf.nn.leaky_relu)
startScalarTraining(epochs=200)


INFO:tensorflow:Summary name network/LAYER-1/kernel:0-grad is illegal; using network/LAYER-1/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/bias:0-grad is illegal; using network/LAYER-1/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/kernel:0-grad is illegal; using network/LAYER-Last/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/bias:0-grad is illegal; using network/LAYER-Last/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/kernel:0-grad is illegal; using network/LAYER-1/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-1/bias:0-grad is illegal; using network/LAYER-1/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/kernel:0-grad is illegal; using network/LAYER-Last/kernel_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-Last/bias:0-grad is illegal; using network/LAYER-Last/bias_0-grad instead.
INFO:tensorflow:Summary name network/LAYER-2/kernel:0-grad is il

In [13]:
# Using TensroBoardX in Pytorch
import numpy as np
from tensorboardX import SummaryWriter

In [15]:
def generateSingleHiddenMLP(nHiddenNeurons):
    return nn.Sequential(nn.Linear(2,nHiddenNeurons), nn.ReLU(), nn.Linear(nHiddenNeurons, 2))
trainx = torch.from_numpy(X_tr).float()
valx = torch.from_numpy(X_val).float()
trainy = torch.from_numpy(Y_tr)
valy = torch.from_numpy(Y_val)

tLog, vLog = SummaryWriter("../log/train_pytorch"), SummaryWriter("../log/val_pytorch")

model1 = generateSingleHiddenMLP(6)
print(trainx.type(), trainy.type())

torch.FloatTensor torch.LongTensor


In [16]:
def trainingRoutine(net, dataset, nIters, gpu):
    # organize the data
    trainData, trainLabels, valData, valLabels = dataset
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
    
    # use the flag
    if gpu:
        trainData, trainLabels = trainData.cuda(), trainLabels.cuda()
        valData, valLabels = valData.cuda(), valLabels.cuda()
        net = net.cuda() # the network parameters also need to be on the gpu
    
    for i in range(nIters):
        # Forward pass
        trainOutput = net(trainData)
        trainLoss = criterion(trainOutput, trainLabels)
        # Backward pass and optimization
        trainLoss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        #Once very 100 iterations, log values
        if i%100==0:
            # compute the accuracy of the perdiction
            trainPrediction = trainOutput.cpu().detach().argmax(dim=1)
            trainAccuracy = (trainPrediction.numpy()==trainLabels.numpy()).mean()
            # Now for the validation set
            valOutput = net(valData)
            valLoss = criterion(valOutput, valLabels)
            # compute accuracy of the prediction
            valPrediction = valOutput.cpu().detach().argmax(dim=1)
            valAccuracy = (valPrediction.numpy()==valLabels.numpy()).mean()
            
            # 1. Log scalar values (scalar summary)
            trInfo = {'loss': trainLoss.cpu().detach().numpy(), 'accuracy': trainAccuracy}
            
            for tag, value in trInfo.items():
                tLog.add_scalar(tag, value, i+1)
                
            # 2. Log values and gradients of the parameters (histogram summary)
            for tag, value in net.named_parameters():
                tag = tag.replace('.', '/')
                tLog.add_histogram(tag, value.data.cpu().numpy(), i+1)
                tLog.add_histogram(tag+'/grad', value.grad.data.cpu().numpy(), i+1)
        
    net = net.cpu()

In [17]:
dataset = trainx, trainy, valx, valy
gpu = False
gpu = gpu and torch.cuda.is_available()

trainingRoutine(model1, dataset, 1000, gpu)